In [1]:
import os
import torch
import torchvision
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2
from matplotlib import pyplot as plt
import torchvision.transforms as transforms
from datetime import datetime
import numpy as np
import os
import random
import argparse
from PIL import Image
import csv

In [2]:
# dataset definition
class myDataset(Dataset):
    # load the dataset
    def __init__(self,root,transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(root, "imgs"))))


    # get a row at an index
    def __getitem__(self, idx):
        idx = idx -1
        img_path = os.path.join(self.root, "imgs", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        
        self.data = open(os.path.join(self.root, "annotations.csv"))
        data = csv.reader(self.data)
        
        boxes = []
        row = data.__next__()
        
        for x in range(idx+1):
            row = data.__next__()
        
        x1 = int(row[1])
        y1 = int(row[2])
        x2 = int(row[3])
        y2 = int(row[4])
        label = int(row[5])
        boxes.append([x1, y1, x2, y2])
        
        self.data.close()
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.tensor([label], dtype=torch.int64)
        
        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target
    
    def __len__(self):
        return len(self.imgs)

In [3]:
import detection.transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.Normalize())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [4]:
model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn()

In [5]:
import detection.utils as utils
dataset = myDataset('db_lisa_tiny', get_transform(train=True))
train,test=torch.utils.data.random_split(dataset, [700, 200], generator=torch.Generator().manual_seed(42))
trainData_loader = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True, collate_fn=utils.collate_fn)
testData_loader = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True, collate_fn=utils.collate_fn)

In [6]:
test_img, test_target = dataset.__getitem__(400)
print(test_target)

{'boxes': tensor([[286., 237., 341., 302.]]), 'labels': tensor([4]), 'image_id': tensor([399]), 'area': tensor([3575.])}


In [7]:
model([test_img], [test_target])

{'loss_classifier': tensor(4.5104, grad_fn=<NllLossBackward0>),
 'loss_box_reg': tensor(0.0080, grad_fn=<DivBackward0>),
 'loss_objectness': tensor(0.6899, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(0.0037, grad_fn=<DivBackward0>)}

In [8]:
test_target

{'boxes': tensor([[286., 237., 341., 302.]]),
 'labels': tensor([4]),
 'image_id': tensor([399]),
 'area': tensor([3575.])}

In [9]:
#@todo Fix Hardcoded params
parameters = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
            parameters,
            lr=0.01,
            momentum=1,
            weight_decay=1,
        )

for i in range(5):
    for images, targets in trainData_loader:
        print("flag1")
        result = model(images, targets)
        print("flag2")
    #with torch.cuda.amp.autocast(enabled=scaler is not None):
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()

flag1
flag2


KeyboardInterrupt: 

In [ ]:
ds = myDataset('/home/semone/db_lisa_tiny')

In [ ]:
ds.imgs[0]

In [ ]:
ds.__getitem__(1)

In [ ]:
train,test=torch.utils.data.random_split(dataset, [700, 200], generator=torch.Generator().manual_seed(42))

In [ ]:
a.__getitem__(7)

In [ ]:
len(dataset)

In [ ]:
test_img

In [ ]:
x = "190"
y = int(x)

In [ ]:
y

In [ ]:
import sys
!{sys.executable} -m pip install sgmllib3k

In [ ]:
import detection.transforms as T

In [ ]:
dataset.__getitem__(3)